# Web Scraping METROCUADRADO página individual

## Librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

### Selección de grilla y extracción

Código base

In [13]:
driver = webdriver.Chrome()
driver.get('https://www.metrocuadrado.com/inmueble/venta-casa-chia-na-3-habitaciones-3-banos-2-garajes/11369-M4772309')
# Get the page source after interactions
page_source = driver.page_source
# Parse the page source with Beautiful Soup
soup = BeautifulSoup(page_source, 'lxml')

driver.quit()

house_grill = soup.find('div', class_ = 'page-container')

info1_list = []

caracteristica_list = []

estrato = int(house_grill.find('ul', class_ = 'list-feature-detail').find_all('h2', class_ = 'card-text')[3].text.replace('Estrato',''))
info1_list.append(estrato)

descripcion = house_grill.find('p', class_ = 'card-text').text
info1_list.append(descripcion)

info_principal = house_grill.find('div', class_ = 'card-details').find_all('div', class_ = 'col-lg-3')

for info in info_principal:
    info1_list.append(info.find('p', class_ = 'card-text').text)

gastos_notariales = house_grill.find('p', class_ = 'notary-fees-simulator-container__info__price').text
info1_list.append(gastos_notariales)

caracteristicas = house_grill.find_all('div', class_ = 'featureacordion')[0].find_all('div', class_ = 'col-md-3')

for caracteristica in caracteristicas:
    caracteristica_list.append(caracteristica.text)

info1_list.append(', '.join(map(str, caracteristica_list)))

# Ejecución

In [24]:
def scrape_property_info(url):
    """
    Scrapes property information from a given Metrocuadrado listing URL.

    Parameters
    ----------
    url : str
        The URL of the Metrocuadrado property listing page.

    Returns
    -------
    list
        A list containing various details about the property, such as 
        estrato, description, principal information, notarial fees, 
        and characteristics.

    Examples
    --------
    >>> info = scrape_property_info('https://www.metrocuadrado.com/inmueble/venta-casa-chia-na-3-habitaciones-3-banos-2-garajes/11369-M4772309')
    >>> print(info)
    [3, 'Charming house in Chia...', '$680,000,000', ...]
    """
    # Initialize the Chrome WebDriver
    driver = webdriver.Chrome()
    
    # Load the specified URL
    driver.get(url)
    
    # Get the page source after interactions
    page_source = driver.page_source
    
    # Parse the page source with Beautiful Soup
    soup = BeautifulSoup(page_source, 'lxml')
    
    # Close the WebDriver
    driver.quit()
    
    # Find the main container with the property information
    house_grill = soup.find('div', class_='page-container')
    
    # Initialize an empty list to store the property details
    info1_list = []
    caracteristica_list = []
    
    # Extract the 'estrato' information and append it to the list
    estrato = int(house_grill.find('ul', class_='list-feature-detail').find_all('h2', class_='card-text')[3].text.replace('Estrato', ''))
    info1_list.append(estrato)
    
    # Extract the property description and append it to the list
    descripcion = house_grill.find('p', class_='card-text').text.replace('\n', '')
    info1_list.append(descripcion)
    
    # Extract the main property details and append each to the list
    info_principal = house_grill.find('div', class_='card-details').find_all('div', class_='col-lg-3')
    for info in info_principal:
        info1_list.append(info.find('p', class_='card-text').text)
    
    # Extract notarial fees and append to the list
    gastos_notariales = house_grill.find('p', class_='notary-fees-simulator-container__info__price').text
    info1_list.append(gastos_notariales)
    
    # Extract additional property characteristics and append each to the list
    caracteristicas = house_grill.find_all('div', class_='featureacordion')[0].find_all('div', class_='col-md-3')
    for caracteristica in caracteristicas:
        caracteristica_list.append(caracteristica.text)

    info1_list.append(', '.join(map(str, caracteristica_list)).replace(',','-').replace('&Aacute;', 'Área'))

    # Add link page
    info1_list.append(url)
    
    # Return the list of property details
    return info1_list


## Web Scraping individual

In [22]:
data_principal = pd.read_csv(r'D:\visualization\web_scraping_metrocuadrado\data\metrocuadrado_principal.csv')

In [26]:
info_out = []
url_list = list(data_principal.Link)

total_paginas = len(url_list)  # Número total de páginas

for i, url in enumerate(url_list):

    try:
    # Se almacena data recolectada
        info_out.append(scrape_property_info(url))
    except:
        print(f'Link no scrapeable: {url}')
    
    # Calcular cuántas páginas quedan por procesar
    paginas_restantes = total_paginas - (i + 1)
    print(f"Quedan {paginas_restantes} páginas por procesar.")
    
    # Espera de 5 segundos antes de la siguiente iteración, a menos que sea la última página
    if paginas_restantes > 0:
        time.sleep(10)

df = pd.DataFrame(info_out, columns=[
    'Estrato',
    'Descripción',
    'Codigo',
    'Barrio',
    'Precio',
    'Antiguedad',
    'Area_construida',
    'Area_privada',
    'Valor_administracion',
    'Parqueaderos',
    'Gastos_notariales',
    'Caracteristicas',
    'Link'
])

df.to_csv('metrocuadrado_individual.csv',index= False,sep='|')

Quedan 499 páginas por procesar.
Quedan 498 páginas por procesar.
Quedan 497 páginas por procesar.
Quedan 496 páginas por procesar.
Quedan 495 páginas por procesar.
Quedan 494 páginas por procesar.
Link no scrapeable: https://www.metrocuadrado.com/inmueble/arriendo-casa-cajica-oikos-palos-verdes-3-habitaciones-4-banos-3-garajes/3038-M4964052
Quedan 493 páginas por procesar.
Link no scrapeable: https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-monaco-3-habitaciones-5-banos-3-garajes/2748-M2467070
Quedan 492 páginas por procesar.
Quedan 491 páginas por procesar.
Link no scrapeable: https://www.metrocuadrado.com/inmueble/arriendo-casa-sopo-incluye-yerbabuena-na-4-habitaciones-5-banos-4-garajes/3155-M4963513
Quedan 490 páginas por procesar.
Quedan 489 páginas por procesar.
Link no scrapeable: https://www.metrocuadrado.com/inmueble/venta-casa-bogota-portales-del-norte-4-habitaciones-3-banos-1-garajes/16205-M4697583
Quedan 488 páginas por procesar.
Quedan 487 páginas por proces

In [20]:
info_out[0][-2]

'Apto para niños/ Aire acondicionado/ Alarma/ Citófonos/ Comedor auxiliar/ Tipo de Cortinas persianas/ Cuarto de servicio/ Estudio o biblioteca/ Hall de Alcobas/ Jacuzzi/ Puertas de seguridad/ Zona de lavanderia/ Tipo instalación de gas propano/ Número de Closets 4/ Número de líneas teléfonicas 1/ Se Permite Fumar/ Tipo de calentador gas/ Tipo de Casa condominio/ Tipo de Cocina lineal/ Tipo comedor sala comedor/ Depósito 1/ Tipo de estufa gas/ Tipo de acabado piso porcelanato/ Tipo de piso en alcobas porcelanato/ Tipo de piso en comedor porcelanato/ Tipo de piso en estudio porcelanato/ Tipo de piso en sala porcelanato/ Vista exterior/ &Aacute;rea del lote 126/ Área Terraza/Balcón 20/ Acceso para discapacitados/ Circuito cerrado de TV/ Cuarto de escoltas/ Parqueadero cubierto/ Jardín/ Piscina/ Vista panorámica/ Conjunto cerrado/ Número de Ascensores 0/ Número de niveles 3/ Terraza/Balcón terraza/ Tipo de parqueadero propio/ Vigilancia 24hrs/ Cancha(s) de Basket/ Cancha(s) de Fútbol/ Can

In [61]:
info_out

[]

In [43]:
# df = pd.DataFrame(info_out, columns=[
#     'Estrato',
#     'Descripción',
#     'Codigo',
#     'Barrio',
#     'Precio',
#     'Antiguedad',
#     'Area_construida',
#     'Area_privada',
#     'Valor_administracion',
#     'Parqueaderos',
#     'Gastos_notariales',
#     'Caracteristicas',
#     'Link'
# ])

# df.to_csv('metrocuadrado_individual_parte_1.csv',index= False)

In [57]:
info_out[0]

[3,
 'Te presento imponente, moderna e iluminada casa con apartamento independiente dentro de apetecido conjunto del sector, con muy buena distribución de espacios, con un area lote de 126m2 y un area construida de 350m2, consta de sala, comedor, cocina integral mixta, zona de ropas, estacionamiento cubierto para dos vehículos, terraza interior, sala de tv, piscina, zona bbq, jacuzzi, cuatro habitaciones , 5 baños, 2  balcones, pisos en porcelanato, hermosos jardines, zonas verdes y perfectos acabados, el conjunto ofrece portería, salón social, amplios accesos y vías pavimentadas, estricta vigilancia y hermosos jardines, el sector es fresco, tranquilo y arborizado, cercano a parques, supermercados, centros recreacionales, zonas de comercio, restaurantes, droguerías, estación de servicio, transporte publico y vías principales. \n\n',
 '16741-M4813781',
 'SIERRA NEVADA',
 '$700.000.000',
 'Entre 10 y 20 años',
 '350 m²',
 '126 m²',
 '$160.000',
 '4',
 '$26.498.861',
 'https://www.metrocu